In [5]:
# ========= Cell 1: 填写任务信息 =========
from datetime import datetime
import ipynbname
task_name = "转为CSV"
notebook_name = "barrio_population_calculate2.0.ipynb"  # 不带扩展名
notebook_path = "CASA0004\barrio_population_calculate2.0.ipynb"  # 完整路径
dataset = "2016 JSON & 2016 barrio with pop density"
code_version = "v1.2_G, (2 part of geo cleaning)"
input_dir = r"E:\Dissertation\CASA0004\barrio_pop_density_clenaed.geojson" #这个文件夹里是带着人口密度的，清洗过后的正常的barrio geojson
streetgeojson_dir = r"E:\Dissertation\streetNetWork.geojson"    # 这个是街道网络
estrageojson_dir = r"E:\Dissertation\estratificacion(social_classs).geojson" # 这个是街道网络社会经济分层
output_dir = r"E:\Dissertation\CASA0004\barrio_pop_density_street_estra_clenaed_reproject.geojson"   # 输出文件夹
note = "0.数据以barrio为每一行进行组织，再进行前确保每个图层都位于EPSG4684坐标下。1.每个barrio新建6列记载estra信息的列，分别为0-6，这个信息能在geojson的\"ESTRATO\"这一栏找到；每一列赋值为barrio中包含或重合某类menzana的面积占比；2.计算barrio中所有不同种类街道长度，并且求出总长。街道种类记载在\"MVITipo\"字段中，计算出总长后再求不同类型街道长度占比。"

In [ ]:
# ========= Cell 2: 主任务（初始版本） =========
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon, MultiPolygon
from shapely.ops import unary_union

import time
start_time = time.time()

# ========== 1. 读取数据 ==========
barrio_gdf = gpd.read_file(input_dir).to_crs(epsg=4684)
street_gdf = gpd.read_file(streetgeojson_dir).to_crs(epsg=4684)
estra_gdf = gpd.read_file(estrageojson_dir).to_crs(epsg=4684)

print("数据读取完成：")
print(f"barrio: {len(barrio_gdf)} 条")
print(f"street: {len(street_gdf)} 条")
print(f"estra: {len(estra_gdf)} 条")

# ========== 2. 计算 estra 占比 ==========
# 给每个 barrio 添加 estrato_0 - estrato_6
for e in range(0, 7):
    barrio_gdf[f"estrato_{e}"] = 0.0

# 确保 estra_gdf 的字段正确
if "ESTRATO" not in estra_gdf.columns:
    raise ValueError("❌ estratificacion 数据缺少字段 'ESTRATO'")

# 遍历每个 barrio，计算不同 estrato 的面积占比
for idx, barrio in barrio_gdf.iterrows():
    barrio_geom = barrio.geometry
    if barrio_geom is None or barrio_geom.is_empty:
        continue

    # 和 estra 叠加
    inter = gpd.overlay(
        gpd.GeoDataFrame(geometry=[barrio_geom], crs=4684),
        estra_gdf,
        how="intersection"
    )

    if inter.empty:
        continue

    inter["area"] = inter.geometry.area
    total_area = inter["area"].sum()

    if total_area > 0:
        for e in range(0, 7):
            area_e = inter.loc[inter["ESTRATO"] == e, "area"].sum()
            barrio_gdf.at[idx, f"estrato_{e}"] = area_e / total_area

print("✅ estrato 占比计算完成")

# ========== 3. 计算街道长度及比例 ==========
if "MVITipo" not in street_gdf.columns:
    raise ValueError("❌ street 数据缺少字段 'MVITipo'")

# 先计算每个 barrio 内街道总长
barrio_gdf["street_total_length"] = 0.0

# 统计每种街道类型的长度
street_types = street_gdf["MVITipo"].unique()
for stype in street_types:
    barrio_gdf[f"street_len_{stype}"] = 0.0
    barrio_gdf[f"street_ratio_{stype}"] = 0.0

# 遍历街道，按 barrio 分配长度
for idx, barrio in barrio_gdf.iterrows():
    barrio_geom = barrio.geometry
    if barrio_geom is None or barrio_geom.is_empty:
        continue

    inter = gpd.overlay(
        gpd.GeoDataFrame(geometry=[barrio_geom], crs=4684),
        street_gdf,
        how="intersection"
    )
    if inter.empty:
        continue

    inter["len"] = inter.geometry.length
    total_len = inter["len"].sum()

    barrio_gdf.at[idx, "street_total_length"] = total_len

    if total_len > 0:
        for stype in street_types:
            len_st = inter.loc[inter["MVITipo"] == stype, "len"].sum()
            barrio_gdf.at[idx, f"street_len_{stype}"] = len_st
            barrio_gdf.at[idx, f"street_ratio_{stype}"] = len_st / total_len

print("✅ 街道长度占比计算完成")

# ========== 4. 保存 ==========
barrio_gdf.to_file(output_dir, driver="GeoJSON")

duration = time.time() - start_time
status = "success"
print(f"✅ 任务完成，结果已保存到 {output_dir}, 用时 {duration:.1f} 秒")


数据读取完成：
barrio: 3558 条
street: 139107 条
estra: 43765 条


C:\Users\wengqc\AppData\Local\Temp\ipykernel_25952\988639055.py:45: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  inter["area"] = inter.geometry.area
C:\Users\wengqc\AppData\Local\Temp\ipykernel_25952\988639055.py:45: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  inter["area"] = inter.geometry.area
C:\Users\wengqc\AppData\Local\Temp\ipykernel_25952\988639055.py:45: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  inter["area"] = inter.geometry.area
C:\Users\wengqc\AppData\Local\Temp\ipykernel_25952\988639055.py:45: UserWarning: Geometry is in a geographic CRS. Results from 'area' are l

✅ estrato 占比计算完成


c:\Users\wengqc\AppData\Local\Programs\Python\Python312\Lib\site-packages\geopandas\tools\overlay.py:358: UserWarning: `keep_geom_type=True` in overlay resulted in 81 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  result = _collection_extract(result, geom_type, keep_geom_type_warning)
c:\Users\wengqc\AppData\Local\Programs\Python\Python312\Lib\site-packages\geopandas\tools\overlay.py:358: UserWarning: `keep_geom_type=True` in overlay resulted in 27 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  result = _collection_extract(result, geom_type, keep_geom_type_warning)
c:\Users\wengqc\AppData\Local\Programs\Python\Python312\Lib\site-packages\geopandas\tools\overlay.py:358: UserWarning: `keep_geom_type=True` in overlay resulted in 14 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  result = _

✅ 街道长度占比计算完成
✅ 任务完成，结果已保存到 E:\Dissertation\CASA0004\barrio_pop_density_street_estra_clenaed.geojson, 用时 2443.3 秒


In [7]:
# ========= Cell 2.1: 主任务（严格CRS + 保留所有几何 + 进度条） =========
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon, MultiPolygon
import time
from tqdm import tqdm

start_time = time.time()

# ========== 1. 读取并投影数据 ==========
# Bogotá 常用投影：MAGNA-SIRGAS / Colombia Bogota (EPSG:3116)
target_crs = "EPSG:3116"

barrio_gdf = gpd.read_file(input_dir).to_crs(target_crs)
street_gdf = gpd.read_file(streetgeojson_dir).to_crs(target_crs)
estra_gdf = gpd.read_file(estrageojson_dir).to_crs(target_crs)

print("数据读取并投影完成：")
print(f"barrio: {len(barrio_gdf)} 条")
print(f"street: {len(street_gdf)} 条")
print(f"estra: {len(estra_gdf)} 条")

# ========== 2. 计算 estra 占比 ==========
for e in range(0, 7):
    barrio_gdf[f"estrato_{e}"] = 0.0

if "ESTRATO" not in estra_gdf.columns:
    raise ValueError("❌ estratificacion 数据缺少字段 'ESTRATO'")

print("开始计算 estrato 占比 ...")
for idx, barrio in tqdm(barrio_gdf.iterrows(), total=len(barrio_gdf), desc="Estrato loop"):
    barrio_geom = barrio.geometry
    if barrio_geom is None or barrio_geom.is_empty:
        continue

    inter = gpd.overlay(
        gpd.GeoDataFrame(geometry=[barrio_geom], crs=target_crs),
        estra_gdf,
        how="intersection",
        keep_geom_type=False
    )

    if inter.empty:
        continue

    inter["area"] = inter.geometry.area
    total_area = inter["area"].sum()

    if total_area > 0:
        for e in range(0, 7):
            area_e = inter.loc[inter["ESTRATO"] == e, "area"].sum()
            barrio_gdf.at[idx, f"estrato_{e}"] = area_e / total_area

print("✅ estrato 占比计算完成")

# ========== 3. 计算街道长度及比例 ==========
if "MVITipo" not in street_gdf.columns:
    raise ValueError("❌ street 数据缺少字段 'MVITipo'")

barrio_gdf["street_total_length"] = 0.0
street_types = street_gdf["MVITipo"].unique()

for stype in street_types:
    barrio_gdf[f"street_len_{stype}"] = 0.0
    barrio_gdf[f"street_ratio_{stype}"] = 0.0

print("开始计算街道长度及比例 ...")
for idx, barrio in tqdm(barrio_gdf.iterrows(), total=len(barrio_gdf), desc="Street loop"):
    barrio_geom = barrio.geometry
    if barrio_geom is None or barrio_geom.is_empty:
        continue

    inter = gpd.overlay(
        gpd.GeoDataFrame(geometry=[barrio_geom], crs=target_crs),
        street_gdf,
        how="intersection",
        keep_geom_type=False
    )
    if inter.empty:
        continue

    inter["len"] = inter.geometry.length
    total_len = inter["len"].sum()
    barrio_gdf.at[idx, "street_total_length"] = total_len

    if total_len > 0:
        for stype in street_types:
            len_st = inter.loc[inter["MVITipo"] == stype, "len"].sum()
            barrio_gdf.at[idx, f"street_len_{stype}"] = len_st
            barrio_gdf.at[idx, f"street_ratio_{stype}"] = len_st / total_len

print("✅ 街道长度占比计算完成")

# ========== 4. 保存 ==========
barrio_gdf.to_file(output_dir, driver="GeoJSON")

duration = time.time() - start_time
print(f"✅ 任务完成，结果已保存到 {output_dir}, 用时 {duration/60:.1f} 分钟")


数据读取并投影完成：
barrio: 3558 条
street: 139107 条
estra: 43765 条
开始计算 estrato 占比 ...


Estrato loop: 100%|██████████| 3558/3558 [21:06<00:00,  2.81it/s]


✅ estrato 占比计算完成
开始计算街道长度及比例 ...


Street loop: 100%|██████████| 3558/3558 [20:08<00:00,  2.95it/s]


✅ 街道长度占比计算完成
✅ 任务完成，结果已保存到 E:\Dissertation\CASA0004\barrio_pop_density_street_estra_clenaed_reproject.geojson, 用时 41.9 分钟


In [6]:
# ===== 记录日志 =====
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
def append_log(task_name, dataset, code_version, input_dir, output_dir, status, duration, note):
    repo_dir = 'E:\Dissertation\CASA0004'
    log_path = f"{repo_dir}/operation_log.md"

    # 写入日志
    with open(log_path, "a", encoding="utf-8") as f:
        f.write(f"**任务名称**: {task_name}\n")
        f.write(f"**任务文件**: {notebook_name}\n")
        f.write(f"**文件路径**: {notebook_path}\n")
        f.write(f"**数据集**: {dataset}\n")
        f.write(f"**代码版本**: {code_version}\n")
        f.write(f"**输入目录**: {input_dir}\n")
        f.write(f"**输出目录**: {output_dir}\n")
        f.write(f"**状态**: {status}\n")
        f.write(f"**耗时**: {duration}\n")
        f.write(f"**备注**: {note}\n")
        f.write(f"**记录时间**: {current_time}\n\n")
        f.write("================分割线================\n\n")

    print("✅ 日志写入完成")
status="完成，初始版本投影有误，街长与面积比例是正确，但是可能不是实际值reproject版本应该会好"
duration = "40min"
append_log(task_name, dataset, code_version, input_dir, output_dir, status, duration, note)

✅ 日志写入完成


<>:4: SyntaxWarning: invalid escape sequence '\D'
<>:4: SyntaxWarning: invalid escape sequence '\D'
C:\Users\wengqc\AppData\Local\Temp\ipykernel_25952\1264914100.py:4: SyntaxWarning: invalid escape sequence '\D'
  repo_dir = 'E:\Dissertation\CASA0004'
